# Initial Setups

## autoreload

In [1]:
# %load_ext autoreload
# %autoreload 2

## dir setup

In [2]:
from dotenv import load_dotenv, find_dotenv
import os

this_file_path = os.path.dirname(os.path.abspath("__file__"))
# if script (not notebook)...
# project_dir = os.path.join(os.path.dirname(__file__), os.pardir)

# project directory
project_dir = os.path.join(this_file_path, os.pardir)

# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

True

In [36]:
raw_data_dir = os.path.join(project_dir, os.environ.get("RAW_DATA_DIR"))
processed_data_dir = os.path.join(project_dir, os.environ.get("PROCESSED_DATA_DIR"))
interim_data_dir = os.path.join(project_dir, os.environ.get("INTERIM_DATA_DIR"))
wordvecs_data_dir = os.path.join(project_dir, os.environ.get("WORDVECS_DATA_DIR"))
figures_dir = os.path.join(project_dir, os.environ.get("FIGURES_DIR"))
reports_dir = os.path.join(project_dir, os.environ.get("REPORTS_DIR"))
cv_dir = os.path.join(project_dir, os.environ.get("CV_DIR"))
models_dir = os.path.join(project_dir, os.environ.get("MODELS_DIR"))

## imports

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
# import sweetviz as sv
# import utils

In [5]:
# pd.set_option('max_rows', 100)

In [25]:
from tqdm import tqdm
tqdm.pandas()

/Users/hitoshinagano/.virtualenvs/tensorflow2/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [37]:
import spacy

port_nlp_fn = 'nilc50skip'
port_nlp_fp = os.path.join(wordvecs_data_dir, port_nlp_fn) 
port_nlp = spacy.load(port_nlp_fp)

espa_nlp_fn = 'suc30fast'
espa_nlp_fp = os.path.join(wordvecs_data_dir, espa_nlp_fn) 
espa_nlp = spacy.load(espa_nlp_fp)

### item_data.jl.gz
Note there are titles with the same texts, but with different item_id's.<br>
check with `item_data[item_data.duplicated(subset = ['title'], keep = False)].sort_values('title')`

Run this to reprocess a new `item_data.pkl` in `processed_data_dir`, for example with new embeddings (more dimensions on pre-trained, or custom embeddings)

In [39]:
item_data_fn = 'item_data.jl.gz'
item_data = pd.read_json(os.path.join(raw_data_dir, item_data_fn), lines=True)

#### item_domain used in the scoring

In [ ]:
item_domain = item_domain[['item_id', 'domain_id']]
item_domain_fn = 'item_domain.pkl'
item_domain_fp = os.path.join(processed_data_dir, item_domain_fn)
item_domain.to_pickle(item_domain_fp)

* MLB: Brasil
* MLM: Other countries

In [30]:
item_data.domain_id.str[:3].value_counts()

MLB    1723216
MLM     378210
Name: domain_id, dtype: int64

In [40]:
item_data['title_lower'] = item_data.title.str.lower()
item_data['lang_domain'] = item_data.domain_id.str[:3].replace({'MLM': 'es', 'MLB': 'pt'})

In [41]:
item_data_unique = item_data[['title_lower', 'lang_domain']].drop_duplicates()

In [42]:
def emb_vectors(x):
    if x.lang_domain == 'pt':
        v = port_nlp(x.title_lower) 
    else:
        v = espa_nlp(x.title_lower)
        
    v_vector = v.vector
    vector_norm = v.vector_norm
    if vector_norm == 0:
        return np.zeros(v_vector.shape, dtype = np.float16)
    else:
        return (v_vector / vector_norm).astype(np.float16)

In [44]:
item_data_unique['title_embs'] = item_data_unique.progress_apply(emb_vectors, axis = 1)

100%|██████████| 1925800/1925800 [05:17<00:00, 6071.83it/s]


In [49]:
item_data = pd.merge(item_data, item_data_unique, how = 'left')

In [51]:
item_data_fn = 'item_data.pkl'
item_data_fp = os.path.join(processed_data_dir, item_data_fn)
item_data.to_pickle(item_data_fp)

In [69]:
item_data = item_data[['item_id', 'lang_domain', 'title_embs']]

item_data_embs_only_fn = 'item_data_embs_only.pkl'
item_data_embs_only_fp = os.path.join(processed_data_dir, item_data_embs_only_fn)
item_data.to_pickle(item_data_embs_only_fp)

#### maybe cluster some of these domains into larger domains (smaller ones)? 

In [48]:
item_data.groupby('domain_id').item_id.count().sort_values().head(20)

domain_id
MLM-GAME_CONSOLE_CAMERA_MOUNTS         1
MLM-GOAL_NETS                          1
MLM-GOLF_BAGS                          1
MLM-GOLF_CLUBS_PUTTERS                 1
MLM-GONIOMETERS                        1
MLM-GPS_CASES_AND_COVERS               1
MLM-GRADUATED_CYLINDERS                1
MLM-GRANOLA                            1
MLM-GRASS_PAVERS                       1
MLM-GROOMING_TABLES                    1
MLM-GROUND_RESISTANCE_METERS           1
MLM-STACKABLE_BINS                     1
MLM-HABERDASHERY_FABRIC_FLOWERS        1
MLM-HABERDASHERY_FRINGES               1
MLB-INDUSTRIAL_SILOS                   1
MLM-ALL_TERRAIN_VEHICLE_DRIVE_BELTS    1
MLM-STOLES                             1
MLM-GLASS_SAFETY_FILMS                 1
MLM-AUTOMOTIVE_TRASH_BAGS              1
MLM-ALARM_REMOTE_CONTROLS              1
Name: item_id, dtype: int64

In [47]:
item_data.groupby('domain_id').item_id.count().sort_values().tail(20)

domain_id
MLB-MOTORCYCLES                            10155
MLB-INDUSTRIAL_AND_COMMERCIAL_EQUIPMENT    10252
MLB-PANTS                                  10309
MLB-SOUVENIRS                              11017
MLB-CELLPHONE_COVERS                       11177
MLB-HEADPHONES                             11328
MLB-VIDEO_GAMES                            12742
MLB-WRISTWATCHES                           13136
MLB-HAIR_TREATMENTS                        13529
MLB-ACTION_FIGURES                         13597
MLM-CARS_AND_VANS                          14165
MLB-SANDALS_AND_FLIP_FLOPS                 15102
MLB-VEHICLE_PARTS                          18727
MLB-DRESSES                                21590
MLB-SUPPLEMENTS                            22351
MLB-T_SHIRTS                               23823
MLB-VEHICLE_ACCESSORIES                    28986
MLB-SNEAKERS                               32636
MLB-CELLPHONES                             38390
MLB-CARS_AND_VANS                          41420
Name: item

## train_dataset.jl.gz

In [9]:
def proc_dataset(df):
    number_of_batches = len(df) // 50
    proc_df = list()
    for df_p in tqdm(np.array_split(df, number_of_batches)):
        if 'item_bought' in df_p:
            df_p = pd.concat([df_p.user_history.apply(pd.Series), df_p.item_bought], axis = 1).stack()
            train_dataset = True
        else:
            df_p = df_p.user_history.apply(pd.Series).stack()
            train_dataset = False
            
        df_p = df_p.apply(pd.Series)
        df_p.reset_index(inplace = True)
        df_p.drop(columns = 'level_1', inplace = True)
        
        if train_dataset:
            new_columns = {0: 'item_bought', 'level_0': 'seq'}
            df_p['event_type'] = df_p.event_type.fillna('buy')
        else:
            new_columns = {'level_0': 'seq'}
        
        df_p.rename(columns = new_columns, inplace = True)
        
        # df_p['timezone'] = df_p.event_timestamp.str[-4:]
        df_p['event_timestamp'] = pd.to_datetime(df_p.event_timestamp.str[:-9])
        df_p['time_diff'] = df_p.groupby('seq').event_timestamp.diff().dt.seconds
        
        # if train_dataset:
        proc_df.append(df_p)
        
    proc_df = pd.concat(proc_df)        
    return proc_df

In [10]:
def read_data(raw_fn = 'train_dataset.jl.gz', processed_fn = 'train_dataset.pkl',
              force_process = False, nrows = None):

    processed_fp = os.path.join(processed_data_dir, processed_fn)
    if os.path.exists(processed_fp) and not force_process:
        processed = pd.read_pickle(processed_fp)
    else:
        raw = pd.read_json(os.path.join(raw_data_dir, raw_fn), lines = True, nrows = nrows)
        raw['len_events'] = raw.user_history.str.len()
        raw.sort_values('len_events', inplace = True)
        raw.drop('len_events', axis = 1, inplace = True)
        processed = proc_dataset(raw)
        processed.to_pickle(os.path.join(processed_data_dir, processed_fn))
        
    if 'item_bought' in processed: 
        processed.item_bought = processed.item_bought.fillna(method = 'backfill').astype(int)
        processed['in_nav'] = processed.item_bought == processed.event_info
    
    return processed

In [11]:
%%time
train = read_data('train_dataset.jl.gz', 'train_dataset.pkl')

CPU times: user 4.43 s, sys: 893 ms, total: 5.32 s
Wall time: 5.42 s


In [13]:
def save_true_labels(df, true_fn = 'true.pkl'):
    true_fp = os.path.join(processed_data_dir, true_fn)
    true_df = df[(df.event_type.isnull()) | (df.event_type == 'buy')]
    true_df = true_df[['seq', 'item_bought']]
    true_df.to_pickle(true_fp)

In [14]:
save_true_labels(train)

### 4% of the events correspond to the item bought

In [12]:
train.in_nav.value_counts(normalize = True)

False    0.959279
True     0.040721
Name: in_nav, dtype: float64

### 30% of the items bought show up in the navigation

In [14]:
train.groupby('seq').in_nav.any().value_counts(normalize = True)

False    0.706116
True     0.293884
Name: in_nav, dtype: float64

In [14]:
train.query("seq == 121")#.time_diff

,seq,item_bought,event_info,event_timestamp,event_type,time_diff,in_nav
864,121,388604,MINOXIDIL,2019-10-13 15:33:41,search,NaN,False
865,121,388604,MINOXIDIL,2019-10-13 17:08:36,search,5695.0,False
866,121,388604,429456,2019-10-13 17:09:04,view,28.0,False
867,121,388604,429456,2019-10-13 17:09:43,view,39.0,False
868,121,388604,429456,2019-10-13 17:10:37,view,54.0,False
869,121,388604,MINOXIDIL,2019-10-13 17:10:38,search,1.0,False
870,121,388604,625179,2019-10-13 17:10:46,view,8.0,False
871,121,388604,MINOXIDIL,2019-10-13 17:10:58,search,12.0,False
872,121,388604,410942,2019-10-13 17:11:45,view,47.0,False
873,121,388604,410942,2019-10-13 17:12:29,view,44.0,False


### 49% of the domain_id of items bought show up in the navigated domain_id's

In [18]:
train = pd.merge(train, item_data[['item_id', 'domain_id']], how = 'left', 
                 left_on = 'item_bought', right_on = 'item_id')

In [19]:
train.rename(columns = {'domain_id': 'bought_domain_id'}, inplace = True)

In [22]:
train = pd.merge(train, item_data[['item_id', 'domain_id']], how = 'left', 
                 left_on = 'event_info', right_on = 'item_id')

In [23]:
train.rename(columns = {'domain_id': 'nav_domain_id'}, inplace = True)

In [25]:
train.drop(['item_id_x', 'item_id_y'], axis = 1, inplace = True)

In [27]:
train['in_nav_domain'] = train.bought_domain_id == train.nav_domain_id

In [34]:
train.groupby('seq').in_nav_domain.any().value_counts(normalize = True)

False    0.506594
True     0.493406
Name: in_nav_domain, dtype: float64

## test_dataset.jl.gz

In [36]:
%%time
test = read_data('test_dataset.jl.gz', 'test_dataset.pkl')

100%|██████████| 3541/3541 [34:21<00:00,  1.72it/s]


CPU times: user 34min 41s, sys: 17.4 s, total: 34min 58s
Wall time: 34min 59s


_____

* test_dataset.jl.gz
* sample_submission.csv